# Implementando um algoritmo de otimização genérico

O objetivo deste tutorial é ilustrar a criação de estruturas para implementar um algoritmo de otimização, que usa modelos `JuMP` como parte de um algoritmo maior.
Vamos continuar o exemplo de otimização estocástica do tutorial anterior.

Algumas ideias apresentadas neste notebook são inspiradas do pacote [SDDP.jl](https://github.com/odow/SDDP.jl).

Vamos continuar usando os mesmos pacotes do notebook anterior.

In [1]:
using JuMP
import Distributions
import HiGHS
import Ipopt

## Dados
Os dados para o problema de venda de tortas são:

In [2]:
using Random
Random.seed!(5);

In [3]:
D = Distributions.TriangularDist(150.0, 250.0, 200.0) # Distribuição da demanda
N = 100 # Número de cenários
Ω = 1:N # Conjunto de cenários
P = fill(1 / N, N); # Probabilidade de cada cenário
d = sort!(rand(D, N)); # Demanda em cada cenário

## Organização

O algoritmo de planos cortantes é genérico: ele pode ser usado para resolver qualquer problema de otimização estocástica em que o segundo estágio é um problema de otimização linear (ou convexo) e onde a decisão de primeiro estágio aparece no segundo por uma restrição do tipo $x_1 = \bar{x}$.

A ideia é, em vez de escrevermos o algoritmo de otimização estocástica para cada problema, escrevermos um algoritmo genérico que possa ser aplicado a qualquer problema de otimização estocástica que se encaixe nesse molde.
Desta forma, um usuário (nós mesmos no futuro!) irá
1. Escrever o modelo de primeiro estágio
2. Escrever o modelo de segundo estágio
3. Indicar como o modelo de primeiro estágio aparece no segundo estágio
e daí passar estas informações para o algoritmo genérico.

## Uma primeira versão

In [4]:
"""
    cutting_planes(first_stage, second_stage, scenarios::Vector, initial_bound;
        P=ones(length(scenarios))/length(scenarios), max_iterations=100, tol=1e-4)

Solves a two-stage stochastic program using Kelley's cutting planes algorithm.

first_stage: a JuMP model for the first stage
second_stage: a JuMP model for the second stage
scenarios: Vector of realizations for each scenario
initial_bound: initial bound on the optimal value

Optional arguments:
P: probability of each scenario
max_iterations: maximum number of cutting planes iterations to perform
tol: absolute and relative tolerance for the optimality gap

returns: lower and upper bounds on the optimal value, the optimal solution of the first stage
"""
function cutting_planes(first_stage, second_stage, scenarios::Vector, initial_bound;
    P=ones(length(scenarios))/length(scenarios), max_iterations=100, tol=1e-4)

    # Steps 1 and 2: Add extra variable to the first stage, with initial bound, and add it to the objective
    @variable(first_stage, θ <= initial_bound)
    @objective(first_stage, Max, -first_stage[:cost] + θ)
    for k in 1:max_iterations # Steps 3 and 11
        # Step 4: Solve the first stage and get new trial point
        optimize!(first_stage)
        xᵏ = value.(first_stage[:x])
        # Step 5: Update upper bound        
        ub = objective_value(first_stage)
        # Step 6: Solve the second stage for each scenario
        ret = [solve_second_stage(second_stage, scenario, xᵏ) for scenario in scenarios]
        # Step 7: average
        vᵏ = sum(p * r.v for (p, r) in zip(P, ret))
        λᵏ = sum(p * r.λ for (p, r) in zip(P, ret))
        # Step 8: lower bound
        lb = -value(first_stage[:cost]) + vᵏ
        # Step 9: convergence check
        # println("$k, $lb, $ub, $(ub - lb); x = $xᵏ")
        if isapprox(lb, ub; atol = tol, rtol = tol)
            return (lb, ub, xᵏ)
        end

        # Step 10: new cut
        c = @constraint(first_stage, first_stage[:θ] <= vᵏ + λᵏ' * (first_stage[:x] .- xᵏ)  )
    end
end

cutting_planes

In [5]:
function solve_second_stage(second_stage, scenario, x̄)
    # Set the scenario
    fix.(second_stage[:s], scenario)
    # Fix the first stage solution
    fix.(second_stage[:x_1], x̄)
    # Solve the second stage
    optimize!(second_stage)
    # Get the optimal value of the second stage
    v = objective_value(second_stage)
    λ = reduced_cost.(second_stage[:x_1])
    return (v=v, λ=λ)
end

solve_second_stage (generic function with 1 method)

O que isso exige dos modelos `first_stage` e `second_stage`?
- Ambos já devem possuir um _solver_ associado;
- O modelo de primeiro estágio
    * deve possuir uma variável de decisão `x`,
    * uma expressão `cost` que é o custo apenas do primeiro estágio (ou seja, sem o $V_2(x) = \theta$)
    * e não pode possuir uma variável chamada `theta`.
- O modelo de segundo estágio
    * deve possuir uma variável de decisão `x_1`,
    * e outra variável `s`, que representa os valores possíveis da incerteza a cada cenário.

## Primeiro teste

In [6]:
first_stage = Model(HiGHS.Optimizer)
set_silent(first_stage)
@variable(first_stage, x >= 0)
@expression(first_stage, cost, 2 * x)
@objective(first_stage, Max, -cost);

In [7]:
second_stage = Model(HiGHS.Optimizer)
set_silent(second_stage)
@variable(second_stage, x_1)
@variable(second_stage, 0 <= y_ω) # Não temos d_ω aqui, pois muda a cada cenário!
@variable(second_stage, s) # Variável extra para representar a demanda
@constraint(second_stage, y_ω <= x_1)
@constraint(second_stage, y_ω <= s)
@objective(second_stage, Max, 5 * y_ω - 0.1 * (x_1 - y_ω) );

In [8]:
cutting_planes(first_stage, second_stage, d, 5*maximum(d); P=P)

(550.6972822537107, 550.7353579603821, 200.0996078873452)

## Melhorando o primeiro estágio

Várias das exigências acima quanto aos modelos podem ser resolvidas, ou, pelo menos, reduzidas.
Vamos tratar, inicialmente, os casos mais fáceis:
- A expressão `cost` do primeiro estágio pode ser obtida diretamente a partir do modelo;
- A variável $\theta$ do primeiro estágio não precisa ter um nome **dentro do modelo** (o que evita conflito);
- O problema poderia ser de minimização.

In [9]:
function cutting_planes2(first_stage, second_stage, scenarios::Vector, initial_bound;
    P=ones(length(scenarios))/length(scenarios), max_iterations=100, tol=1e-4,
    verbose=false)
    # Steps 1 and 2: Add extra variable to the first stage, with initial bound, and add it to the objective
    θ₁ = @variable(first_stage)
    sense = objective_sense(first_stage)
    if sense == JuMP.MAX_SENSE
        set_upper_bound(θ₁, initial_bound)
    else
        set_lower_bound(θ₁, initial_bound)
    end
    obj₁ = JuMP.objective_function(first_stage)
    @objective(first_stage, sense, obj₁ + θ₁)
    for k in 1:max_iterations # Steps 3 and 11
        # Step 4: Solve the first stage and get new trial point
        optimize!(first_stage)
        xᵏ = value.(first_stage[:x])
        # Step 5: Update outer bound        
        outer_bound = objective_value(first_stage)
        # Step 6: Solve the second stage for each scenario
        ret = [solve_second_stage(second_stage, scenario, xᵏ) for scenario in scenarios]
        # Step 7: average
        vᵏ = sum(p * r.v for (p, r) in zip(P, ret))
        λᵏ = sum(p * r.λ for (p, r) in zip(P, ret))
        # Step 8: Inner bound
        inner_bound = value(obj₁) + vᵏ
        # Step 9: convergence check
        verbose && println("Iteration $k: $inner_bound, $outer_bound, $(outer_bound - inner_bound); x = $xᵏ")
        if isapprox(inner_bound, outer_bound; atol = tol, rtol = tol)
            return (inner_bound, outer_bound, xᵏ)
        end

        # Step 10: new cut
        if sense == JuMP.MAX_SENSE
            @constraint(first_stage, θ₁ <= vᵏ + λᵏ' * (first_stage[:x] .- xᵏ) )
        else
            @constraint(first_stage, θ₁ >= vᵏ + λᵏ' * (first_stage[:x] .- xᵏ) )
        end
    end
end

cutting_planes2 (generic function with 1 method)

In [10]:
# Temos que criar novamente o modelo do primeiro estágio, pois ele foi alterado pela função `cutting_planes`
first_stage = Model(HiGHS.Optimizer)
set_silent(first_stage)
@variable(first_stage, x >= 0)
@objective(first_stage, Max, -2*x);

In [11]:
cutting_planes2(first_stage, second_stage, d, 5*maximum(d); P=P)

(550.6972822537107, 550.7353579603821, 200.0996078873452)

## Um exemplo mais complicado: duas variáveis aleatórias

Suponha que você tem que fornecer energia elétrica para uma cidade.
A demanda é aleatória, e você pode comprar energia de duas fontes diferentes.
Uma delas exige que você compre antecipadamente a energia, e a outra permite que você compre energia no momento em que a demanda ocorre.
Enquanto o preço da primeira é dado, a segunda cobra um preço _dinâmico_ pela energia.

O modelo de otimização é

$$ \begin{array}{rl}
\min_{x_1, x_2} &  c_1 x_1^2 + \mathbb{E}\left[ c_2 x_2^2 \right] \\[0.5ex]
\text{s.a.} & x_1 + x_2 \geq d \\
& x_1, x_2 \geq 0
\end{array} $$


In [12]:
# Dados
c1 = 3

μd = 100
σd = 10
μc = 4
σc = 0.5

# Incerteza
NScen = 100
# Demanda e custos
demand = μd .+ σd*randn(NScen)
costs  = μc .+ σc*randn(NScen)
scenarios = collect(zip(demand, costs))

100-element Vector{Tuple{Float64, Float64}}:
 (85.71298746006389, 4.58794857211512)
 (99.71726233515668, 4.004828288044029)
 (101.07323576257558, 3.988730049086219)
 (113.14997170531423, 3.3584943246063337)
 (113.37175648950021, 3.060332885678811)
 (104.9461539711795, 3.949391143256375)
 (82.54644618836011, 4.041514882000625)
 (95.37816147627852, 4.1766782135897245)
 (105.64454607127982, 3.889831192700104)
 (91.1224571533337, 3.969798355683531)
 ⋮
 (96.55385266139284, 4.076329100911172)
 (112.99948459345805, 3.233341371801422)
 (106.37307626316371, 3.4823030464269498)
 (89.61453909291626, 3.7610764464187456)
 (124.02571220218391, 2.99104199670204)
 (96.48144076083325, 4.321783944011804)
 (122.63903171375604, 4.301880371708172)
 (86.55640130775564, 4.928905165504547)
 (94.4326446308811, 3.290662987500371)

Se tentássemos a mesma abordagem do exemplo anterior, a função objetivo do segundo estágio ficaria com um termo **cúbico**, que não é quadrático, e portanto não é aceito pelo HiGHS (mas ainda poderia funcionar com outro solver).

In [13]:
first_stage_energy = Model(Ipopt.Optimizer)
set_silent(first_stage_energy)
@variable(first_stage_energy, x >= 0)
@objective(first_stage_energy, Min, c1*x^2)

second_stage_energy = Model(Ipopt.Optimizer)
set_silent(second_stage_energy)
# Variáveis de decisão
@variable(second_stage_energy, x_1)
@variable(second_stage_energy, 0 <= x_2)
# Variáveis de incerteza: Demanda e custo
@variable(second_stage_energy, s[1:2])
d, c_2 = s
# Restrição e Fobj
@constraint(second_stage_energy, x_1 + x_2 >= d)
@objective(second_stage_energy, Min, c_2 * x_2^2);

In [14]:
cutting_planes2(first_stage_energy, second_stage_energy, scenarios, 0)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



(16924.295061756333, 16923.748180415136, 56.18175192245918)

## Generalizando os cenários

Para resolver o problema acima, vamos generalizar o tratamento dos cenários no algoritmo:
- O usuário irá definir uma função que modifica o problema para adaptar aos dados do cenário;
- O algoritmo irá chamar esta função, que estará registrada em um local específico.

In [15]:
second_stage_energy = Model(HiGHS.Optimizer)
set_silent(second_stage_energy)
# Variáveis de decisão
@variable(second_stage_energy, x_1)
@variable(second_stage_energy, 0 <= x_2)
# Restrição "incompleta"
@constraint(second_stage_energy, demand_eq, x_1 + x_2 >= 0)

function set_demand_and_cost(scenario)
    set_normalized_rhs(demand_eq, scenario.d)
    @objective(second_stage_energy, Min, scenario.cost * x_2^2)
end

second_stage_energy.ext[:set_scenario] = set_demand_and_cost
scenarios3 = [(d=d, cost=cost) for (d, cost) in zip(demand, costs)];

In [16]:
print(second_stage_energy)

Feasibility
Subject to
 demand_eq : x_1 + x_2 >= 0
 x_2 >= 0


In [17]:
set_demand_and_cost(scenarios3[32])
print(second_stage_energy)

Min 3.521331708535423 x_2²
Subject to
 demand_eq : x_1 + x_2 >= 114.6618630138033
 x_2 >= 0


Isto exige modificar a função que resolve cada cenário do problema de segundo estágio.

In [18]:
function solve_second_stage(second_stage, scenario, x̄)
    # Set the scenario
    second_stage.ext[:set_scenario](scenario)
    # Fix the first stage solution
    fix.(second_stage[:x_1], x̄)
    # Solve the second stage
    optimize!(second_stage)
    # Get the optimal value of the second stage
    v = objective_value(second_stage)
    λ = reduced_cost.(second_stage[:x_1])
    return (v=v, λ=λ)
end

solve_second_stage (generic function with 1 method)

In [19]:
first_stage_energy = Model(Ipopt.Optimizer)
set_silent(first_stage_energy)
@variable(first_stage_energy, x >= 0)
@objective(first_stage_energy, Min, c1*x^2);

In [20]:
cutting_planes2(first_stage_energy, second_stage_energy, scenarios3, 0; verbose=true)

Iteration 1: 38726.27889451072, -5.59402998579499e-9, -38726.27889451631; x = 3.414127074921731e-5
Iteration 2: 17196.866851621737, 7412.833724658, -9784.033126963735; x = 49.708596589402724
Iteration 3: 17399.369470646918, 16563.81091999618, -835.5585506507377; x = 64.23507622288952
Iteration 4: 16930.964592968434, 16722.074869463017, -208.88972350541735; x = 56.971836599864666
Iteration 5: 16936.144816245192, 16914.770291150984, -21.374525094208366; x = 54.64846248618314
Iteration 6: 16924.162548402855, 16918.81884275528, -5.343705647574097; x = 55.81014970756685
Iteration 7: 16924.29506279614, 16923.74818552548, -0.5468772706590244; x = 56.18175114118037


(16924.29506279614, 16923.74818552548, 56.18175114118037)

In [21]:
print(first_stage_energy)

Min 3 x² + _[2]
Subject to
 779.0665609155031 x + _[2] >= 38726.305492829604
 385.41045733745693 x + _[2] >= 28942.246072082653
 270.3710955618862 x + _[2] >= 22388.242350449542
 327.8907749155587 x + _[2] >= 25874.13374737561
 346.2902414034795 x + _[2] >= 26901.01072660325
 337.0905068564564 x + _[2] >= 26392.915769918203
 x >= 0
 _[2] >= 0


## Generalizando as decisões de primeiro estágio

Até agora, a decisão de primeiro estágio era unidimensional.
Se fosse um vetor, com nome `x`, o algoritmo que programamos também funcionaria, graças ao _broadcasting_.
Porém, se houver várias variáveis de decisão, ou simplesmente se o vetor de decisão tiver um nome diferente, o algoritmo não funcionará.
Para isso, o ideal seria também aproveitar o mecanismo do dicionário de extensão para que o usuário indique quais variáveis de decisão do primeiro estágio devem ser usadas no segundo estágio.

Aqui, mais uma vez, estamos generalizando o algoritmo para que ele possa ser usado em uma classe maior de problemas, mas a contrapartida é que o usuário (do algoritmo) será responsável por garantir a compatibilidade.
Ou seja, a ordem em que ele irá declarar as variáveis de decisão no dicionário de extensão do primeiro estágio deve ser a mesma em que elas aparecem no `.ext` do segundo estágio.

In [22]:
function solve_second_stage(second_stage, scenario, x̄)
    # Set the scenario
    second_stage.ext[:set_scenario](scenario)
    # Fix the first stage solution
    fix.(second_stage.ext[:x_1], x̄)
    # Solve the second stage
    optimize!(second_stage)
    # Get the optimal value of the second stage
    v = objective_value(second_stage)
    λ = reduced_cost.(second_stage.ext[:x_1])
    return (v=v, λ=λ)
end

solve_second_stage (generic function with 1 method)

In [23]:
function cutting_planes3(first_stage, second_stage, scenarios::Vector, initial_bound;
    P=ones(length(scenarios))/length(scenarios), max_iterations=100, tol=1e-4,
    verbose=false, full_output=false)
    # Steps 1 and 2: Add extra variable to the first stage, with initial bound, and add it to the objective
    θ₁ = @variable(first_stage)
    sense = objective_sense(first_stage)
    if sense == JuMP.MAX_SENSE
        set_upper_bound(θ₁, initial_bound)
    else
        set_lower_bound(θ₁, initial_bound)
    end
    obj₁ = JuMP.objective_function(first_stage)
    @objective(first_stage, sense, obj₁ + θ₁)

    if full_output
        info = []
        cuts = []
    end
    for k in 1:max_iterations # Steps 3 and 11
        # Step 4: Solve the first stage and get new trial point
        optimize!(first_stage)
        xᵏ = value.(first_stage.ext[:x])
        # Step 5: Update outer bound        
        outer_bound = objective_value(first_stage)
        # Step 6: Solve the second stage for each scenario
        ret = [solve_second_stage(second_stage, scenario, xᵏ) for scenario in scenarios]
        # Step 7: average
        vᵏ = sum(p * r.v for (p, r) in zip(P, ret))
        λᵏ = sum(p * r.λ for (p, r) in zip(P, ret))
        # Step 8: Inner bound
        inner_bound = value(obj₁) + vᵏ
        # Step 9: convergence check
        verbose && println("$k, $inner_bound, $outer_bound, $(outer_bound - inner_bound); x = $xᵏ")
        push!(info, (inner_bound=inner_bound, outer_bound=outer_bound, x=xᵏ))
        if isapprox(inner_bound, outer_bound; atol = tol, rtol = tol)
            if full_output
                return (inner_bound, outer_bound, xᵏ, info, cuts)
            else
                return (inner_bound, outer_bound, xᵏ)
            end
        end

        # Step 10: new cut
        if sense == JuMP.MAX_SENSE
            c = @constraint(first_stage, θ₁ <= vᵏ + λᵏ * (first_stage.ext[:x] .- xᵏ)  )
        else
            c = @constraint(first_stage, θ₁ >= vᵏ + λᵏ * (first_stage.ext[:x] .- xᵏ)  )
        end
        push!(cuts, (v=vᵏ, λ=λᵏ, x=xᵏ))
    end
end

cutting_planes3 (generic function with 1 method)

## Exercício:

Adapte os modelos de tortas / energia para esta função de planos cortantes, e faça o gráfico de convergência do algoritmo.